## [Day 20](https://adventofcode.com/2020/day20)

Alright this one looks *quite* difficult. We're given a set of tiles that are supposed to connect with one another in a way that all values along the edges of two adjacent tiles should be identical. The overall image is a square so we can use that fact to our adavantage. We're also allowed to rotate or flip the tiles in any way that would allow them to fit.

So.... I have some thoughts to start out with but not an overall strategy yet:
1. There are 8 orientations a tile can take (dihedral group 4)
2. We will likley want a function that checks two tiles against eachother
3. I think this would be a good chance to work with np.array
4. The ultimate solution will likely be a some branching process where as soon as we find a matching pair, we try each subsequent possiblity and so fourth until we run out of options or find the solution.

In [1]:
import pandas as pd
import numpy as np
import itertools, math
tiles = open('../inputs/d20_check.txt').read().splitlines()
tiles[:15]

['Tile 2311:',
 '..##.#..#.',
 '##..#.....',
 '#...##..#.',
 '####.#...#',
 '##.##.###.',
 '##...#.###',
 '.#.#.#..##',
 '..#....#..',
 '###...#.#.',
 '..###..###',
 '',
 'Tile 1951:',
 '#.##...##.',
 '#.####...#']

In [2]:
# So maybe we'll just start out with making a dictionary of all the isomorphisms 
# of the tiles as np.arrays
orig_tiles = {}
for row in tiles:
    if row[:4] == 'Tile':
        tile_id = row[5:-1]
        new_tile = []
    elif row == '':
        orig_tiles.update({tile_id:np.array(new_tile)})
    else:
        new_tile.append([x for x in row])
orig_tiles.update({tile_id:np.array(new_tile)})

In [3]:
# Testing a smaller one yet to figure out if this is a memory issue:
extra_keys = ['3079', '2473', '2971', '1489', '1171']
for key in extra_keys:
    del orig_tiles[key]
orig_tiles



{'2311': array([['.', '.', '#', '#', '.', '#', '.', '.', '#', '.'],
        ['#', '#', '.', '.', '#', '.', '.', '.', '.', '.'],
        ['#', '.', '.', '.', '#', '#', '.', '.', '#', '.'],
        ['#', '#', '#', '#', '.', '#', '.', '.', '.', '#'],
        ['#', '#', '.', '#', '#', '.', '#', '#', '#', '.'],
        ['#', '#', '.', '.', '.', '#', '.', '#', '#', '#'],
        ['.', '#', '.', '#', '.', '#', '.', '.', '#', '#'],
        ['.', '.', '#', '.', '.', '.', '.', '#', '.', '.'],
        ['#', '#', '#', '.', '.', '.', '#', '.', '#', '.'],
        ['.', '.', '#', '#', '#', '.', '.', '#', '#', '#']], dtype='<U1'),
 '1951': array([['#', '.', '#', '#', '.', '.', '.', '#', '#', '.'],
        ['#', '.', '#', '#', '#', '#', '.', '.', '.', '#'],
        ['.', '.', '.', '.', '.', '#', '.', '.', '#', '#'],
        ['#', '.', '.', '.', '#', '#', '#', '#', '#', '#'],
        ['.', '#', '#', '.', '#', '.', '.', '.', '.', '#'],
        ['.', '#', '#', '#', '.', '#', '#', '#', '#', '#'],
        [

In [4]:
# Store the ids and make an example for some tests:
all_ids = list(orig_tiles.keys())
example_id = all_ids[0] 
test_tile = orig_tiles[example_id]
test_tile

array([['.', '.', '#', '#', '.', '#', '.', '.', '#', '.'],
       ['#', '#', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['#', '.', '.', '.', '#', '#', '.', '.', '#', '.'],
       ['#', '#', '#', '#', '.', '#', '.', '.', '.', '#'],
       ['#', '#', '.', '#', '#', '.', '#', '#', '#', '.'],
       ['#', '#', '.', '.', '.', '#', '.', '#', '#', '#'],
       ['.', '#', '.', '#', '.', '#', '.', '.', '#', '#'],
       ['.', '.', '#', '.', '.', '.', '.', '#', '.', '.'],
       ['#', '#', '#', '.', '.', '.', '#', '.', '#', '.'],
       ['.', '.', '#', '#', '#', '.', '.', '#', '#', '#']], dtype='<U1')

In [5]:
# Record the size of the grid
grid_size = int(math.sqrt(len(orig_tiles)))

In [6]:
# Now find the isomorphism... we need a way to rotate and flip the array.
# Fortunately, these already exist in numpy:
def get_isomorphisms(tile):
    isos = []
    for i in range(4):
        # I believe that I should be copying here since
        # the documentation says that it returns a view of the array
        isos.append(np.rot90(tile.copy(), i))
        isos.append(np.rot90(np.fliplr(tile.copy()), i))
    return isos

tile_isos = {}
for tile_id in list(orig_tiles.keys()):
    tile_isos.update({tile_id:get_isomorphisms(orig_tiles[tile_id])})

Alright, I think this is a good start. Next we're going to want a function that checks two tiles for a match and then one that checks all of the neighbors in a potential solution

In [7]:
# So this function will take in two tiles and an axis argument to check if they can fit together.
# The first argument will be the top/left while the second will be the bottom/right
def check_tiles(t1, t2, axis):
    # Horiszontal comparison
    if axis == 1:
        t1 = t1[:,9]
        t2 = t2[:,0]
    # Vertical comparison
    elif axis == 0:
        t1 = t1[9,:]
        t2 = t2[0,:]
    else:
        raise ValueError('Bad axis svenny')

    return np.array_equal(t1, t2)

check_tiles(test_tile, np.fliplr(test_tile), axis = 1)

True

In [8]:
check_tiles(test_tile, np.flipud(test_tile), axis = 0)

True

In [9]:
check_tiles(test_tile, np.fliplr(test_tile), axis = 0)

False

Now a function that will take in a dictionary of the positions of the tiles we've placed so far and check whether or not it matches the neighbors

In [10]:
# I'm going to fill it left to right, top to bottom so some of these
# checks are unnecessary and we shouldn't worry about key errors

def check_neighbors(tile, pos, d):
    x, y = pos[0], pos[1]
    
    if y > 0:
        if not check_tiles(d[x,y-1], tile, axis = 0):
            return False
#    if y < grid_size-1 and (x,y+1) in d.keys():
#        if not check_tiles(tile, d[x,y+1], axis = 0):
#            return False
    if x > 0:
        if not check_tiles(d[x-1,y], tile, axis = 1):
            return False
#    if x < grid_size-1 and (x+1,y) in d.keys():
#        if not check_tiles(tile ,d[x+1,y], axis = 1):
#            return False
    return True

In [11]:
grid_side = list(range(grid_size))
coords = list(itertools.product(grid_side, grid_side))

# Initialize the position matrix:
def refresh_pos(tile_id, tile):
    new_dict = dict(zip(coords, [None for i in range(len(coords))]))
    new_dict.update({(0,0):(tile_id, tile)})
    return new_dict

So what are the steps involved here:

1. Initialize the positions with an orientation of one of the puzzle pieces
2. Loop through each subsequent orientation of each other piece
3. If we find a match, remove it from the list and call the function again

In [12]:
def try_piece(pos_dict):
    
    # First find the position we're trying to fill:
    for coord in coords:
        if pos_dict[coord] is not None:
            next_coord = coord
            break
            
    # Now we find the potential candidates for the next tile
    non_candidates = []
    for key in pos_dict.keys():
        if pos_dict[key] is not None:
            non_candidates.append(pos_dict[key][0])
    candidates = [tile_id for tile_id in all_ids if tile_id not in non_candidates]
    
    # If there are any left to try:
    if len(candidates) > 0:
        # try each next tile:
        for candidate in candidates:
            # Try each isomorphism of that tile:
            for iso_cand in tile_isos[candidate]:
                if check_neighbors(iso_cand, next_coord, pos_dict):
                    # If we found a piece that can fit, call the function again:
                    next_pos_dict = pos_dict.copy()
                    next_pos_dict.update({next_coord:(candidate, iso_cand)})
                    if type(try_piece(next_pos_dict)) == dict:
                        return try_piece(next_pos_dict)
        # If all fail:
        else:
            return False
    # If we have no candidates, we return the dictionary as a success:
    else:
        return pos_dict

In [ ]:
# Kinda curious how many multi-matches there are...

def find_side_matches(tile_id):
    



In [ ]:
# loop through each of the ids as the start until we break the loop due to success:
for next_id in all_ids:
    print(next_id)
    for next_orient in tile_isos[next_id]:
        
        # Refresh the dictionary:
        pos_dict = refresh_pos(next_id, next_orient)

        # call the function on each successive
        attempt = try_piece(pos_dict)
        if type(attempt) == dict:
            break
#sol1 = attempt[(0,0)]*attempt[(0,grid_size-1)]*attempt[(grid_size-1,0)]*attempt[(grid_size-1,grid_size-1)]
#sol1

In [18]:
8**9

134217728

In [14]:
pos_dict

{(0, 0): ('3079',
  array([['.', '.', '.', '#', '.', '.', '.', '.', '#', '.'],
         ['.', '.', '.', '#', '#', '#', '.', '.', '#', '#'],
         ['.', '.', '.', '.', '#', '.', '.', '.', '#', '#'],
         ['#', '.', '#', '#', '.', '.', '.', '.', '#', '#'],
         ['#', '.', '#', '#', '#', '#', '#', '.', '#', '#'],
         ['#', '.', '#', '#', '.', '.', '#', '.', '#', '#'],
         ['.', '.', '.', '#', '.', '#', '#', '.', '.', '.'],
         ['#', '#', '#', '#', '.', '#', '#', '#', '.', '#'],
         ['.', '.', '.', '.', '#', '#', '#', '.', '#', '.'],
         ['.', '.', '.', '#', '.', '#', '#', '.', '.', '#']], dtype='<U1')),
 (0, 1): None,
 (0, 2): None,
 (1, 0): None,
 (1, 1): None,
 (1, 2): None,
 (2, 0): None,
 (2, 1): None,
 (2, 2): None}

In [22]:
for next_id in all_ids[:1]:
    print(next_id)

2311


In [15]:
list(coords)

[(1, 2), (2, 0), (2, 1), (2, 2)]

In [119]:
x = [1, 2, 3]
y = [2, 5, 6]
[z for z in y if z not in x]

[5, 6]

### Part 2

In [103]:
x = [0,1,2]
x.pop()
x

[0, 1]

In [117]:
type({}) == dict

True

In [122]:
for i in {0:1, 2:44}.values():
    print(i)

1
44
